# ChexNet

ChexNet is a 121-layer dense convolutional neural network that inputs a chest X-ray image and outputs the probability of thoracic abnormality(*originally trained for detecting penumonia*), along with a heatmap localizing the areas of the image most indicative of that abnormality.

DenseNets improve flow of information and gradients through the network, making the optimization of very deep networks tractable. We replace the final fully connected layer with one that has a single output, after which we apply a sigmoid nonlinearity. 

In [20]:
import sys
print(sys.executable)

/media/HHD_2TB/baurai/aditya_pytorch_vm/bin/python3.6


In [16]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

from sklearn.metrics import roc_auc_score
import cv2
import torch
import torchvision
from torch.utils.data import Dataset
import torchvision.transforms as transforms

print("Python version used = ", sys.version)
print("OpenCV version = ", cv2.__version__)
print("PyTorch used = ", torch.__version__)

Python version used =  3.6.9 (default, Oct  9 2020, 08:58:16) 
[GCC 6.3.0 20170516]
OpenCV version =  4.5.1
PyTorch used =  1.7.0+cu101


# Dataset : 

NIH Dataset images available at : "/media/HHD2/NIH/tflow_obj_detection/images"

Metadata for NIH is available at : "/media/HHD2/NIH/tflow_obj_detection/data/Data_Entry_2017_v2020.csv"

FILTERED_METADATA_PATH_NIH : contains the OG csv where nodule cases are marked as 1, and healthy as 0.

In [4]:
DATASET_PATH_NIH = "/media/HHD2/NIH/tflow_obj_detection/images/"
METADATA_PATH_NIH = "/media/HHD2/NIH/tflow_obj_detection/data/Data_Entry_2017_v2020.csv"
FILTERED_METADATA_PATH_NIH = "/media/HHD_2TB/baurai/filtered_metadata/metadata.csv"

1 - Nodule
0 - Healthy

In [5]:
df = pd.DataFrame(pd.read_csv(FILTERED_METADATA_PATH_NIH))

print(f"Shape of the dataframe = {df.shape}")

Shape of the dataframe = (66692, 5)


In [6]:
df.head()

,image_id,label,patient_id,age,gender
0,00000002_000.png,0,2,80,M
1,00000004_000.png,1,4,82,M
2,00000005_000.png,0,5,69,F
3,00000005_001.png,0,5,69,F
4,00000005_002.png,0,5,69,F


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66692 entries, 0 to 66691
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_id    66692 non-null  object
 1   label       66692 non-null  int64 
 2   patient_id  66692 non-null  int64 
 3   age         66692 non-null  int64 
 4   gender      66692 non-null  object
dtypes: int64(3), object(2)
memory usage: 2.5+ MB


In [8]:
def load_image(base_path, image_name) : 
    image_path = os.path.join(base_path, image_name)
    image = cv2.imread(image_path)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [9]:
def resize_image(image, image_size) : 
    image = cv2.resize(image, (image_size[0], image_size[1]), interpolation = cv2.INTER_AREA)
    return image

# Define Config Data Structure

In [12]:
config = {
    "NUM_CLASSES" : 1,
    "CLASS_NAMES" : ["Nodule"],
    "IMAGE_DATA_DIR" : "/media/HHD2/NIH/tflow_obj_detection/images",
    "METADATA_PATH" : "/media/HHD_2TB/baurai/filtered_metadata/metadata.csv",
    "BATCH_SIZE" : 32,
    "CHECKPOINT_PATH" : "/media/HHD_2TB/baurai/saved_models/chexnet"
}

# Read Images & Labels

In [13]:
def PIL_to_tensor(image_path) : 
    """
    input : image path.
    output : tensor
    """
    loader = transforms.Compose([
        transforms.ToTensor()
    ])
    
    image = Image.open(image_path).convert("RGB")
    image = loader(image).unsqueeze(0)
    return image.to(torch.float)

In [14]:
def tensor_to_PIL(tensor) : 
    """
    input : a tensor.
    output : PIL format image
    """
    unloader = transforms.ToPILImage()
    
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = unloader(image)
    return image

In [15]:
class ChestXRayDataset(Dataset) : 
    def __init__(self, data_dir, image_metadata_file_path, transform = None) : 
        """
        data_dir = path to the directory containing images.
        image_metadata_file_path = path to the metadata file(csv) containing images and labels.
        transform = optional parameter. If specified, the required transforms will be applied on the image.
        """
        image_paths = []
        labels = []
        
        image_metadata_file = pd.DataFrame(pd.read_csv(image_metadata_file_path))
        all_image_ids = image_metadata_file.image_id.values
        all_labels = image_metadata_file.label.values
        for (image_id, label) in zip(all_image_ids, all_labels) : 
            full_image_path = os.path.join(data_dir, image_id)
            image_paths.append(full_image_path)
            labels.append(int(label))
        
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __getitem__(self, index) : 
        """
        Upon passing the index, we get the corresponding image and labels.
        """
        loader = transforms.Compose([
            transforms.ToTensor()
        ])
        
        image_path = self.image_paths[index]
        image = PIL_to_tensor(image_path)
        #image = Image.open(image_name).convert("RGB")
        #image = loader(image).unsqueeze(0)
        label = self.labels[index]
        print(label)
        if self.transform is not None : 
            image = self.transform(image)
        return image, torch.tensor(label)
    
    def __len__(self) : 
        """
        Returns total number of images available.
        """
        return len(self.image_paths)

## Compute AUC Method : 

Computes Area Under the Curve (AUC) from prediction scores.
    
Args:
* ground_truth: Pytorch tensor on GPU, shape = [n_samples, n_classes] || true binary labels.
* pred: Pytorch tensor on GPU, shape = [n_samples, n_classes] || can either be probability estimates of the positive class, confidence values, or binary decisions.
    
    
It returns the list of AUROCs of all classes.

In [ ]:
def compute_AUCs(ground_truth, pred) : 
    aucrocs = []
    ground_truth_numpy = ground_truth.cpu().numpy()
    pred_numpu = pred.cpu().numpy()
    for i in range(config["NUM_CLASSES"]) : 
        aucrocs.append(roc_auc_score(ground_truth_numpy[:, i], pred_numpy[:, i]))
    return aurocs

# DenseNet

In [ ]:
class DenseNet121(torch.nn.Module) : 
    def __init__(self, output_size) : 
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained = True)
        num_features = self.densenet121.classifier.in_features
        self.densenet121.classifier = torch.nn.Sequential(
            torch.nn.Linear(num_features, output_size),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x) : 
        x = self.densenet121(x)
        return x

# Design Model Classes

* When performing Normalize, you need to convert to the form of Tensor first.
* The operation of Resize and crop is the operation of the format of PIL Image. Now the paper generally resizes the image to (256, 256) and then randomCrop to (224, and 224).
* Before transforms.TenCrop(224), you have to add transforms.Scale(256), some of your images might be too small to be cropped to 224.

Resize is equivalent to compressing the original image, the approximate shape is not changed, that is, you can see how the image looks.

Crop is a random cut of the image, which may be part of the entire image, where RandomCrop is more commonly used.
The RandomResizedCrop class is also commonly used. In general, it is to do the crop first, then resize to the specified size.

## FiveCrop and TenCrop


In [16]:
def main() : 
    torch.backends.cudnn.benchmark = True
    
    model = DenseNet121(config["NUM_CLASSES"]).cuda()
    model = torch.nn.DataParallel(model).cuda()
    
    if os.path.isfile(config["CHECKPOINT_PATH"]) : 
        print("===> Loading model from checkpoint")
        checkpoint = torch.load(config["CHECKPOINT_PATH"])
        model.load_state_dict(checkpoint["state_dict"])
        print("===> Loaded checkpoint")
    else:
        print("===> No checkpoint found")
    
    normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
    val_dataset = ChestXRayDataset(data_dir = config["IMAGE_DATA_DIR"], image_metadata_file_path = config["METADATA_PATH"], 
                                  transform = transforms.Compose([
                                      transforms.resize(256),
                                      transforms.TenCrop(224),
                                      transforms.lambda(lambda crops : 
                                                        torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.lambda(lambda crops : torch.stack([normalize(crop) for crop in crops]))
                                  ]))
    val_loader = torch.utils.data.DataLoader(dataset = val_dataset, batch_size = config["BATCH_SIZE"], shuffle = False,
                                            num_workers = 8, pin_memory = True)
    
    ground_truth = torch.FloatTensor()
    ground_truth = ground_truth.cuda()
    pred = torch.FloatTensor()
    pred = pred.cuda()
    
    model.eval()
    
    for i, (input_image, target) in enumerate(val_loader) : 
        target = target.cuda()
        ground_truth = torch.cat((ground_truth, target), 0)
        batch_size, n_crops, channels, height, width = input_image.size()
        input_var = torch.autograd.Variable(input_image.view(-1, channels, height, width).cuda(), volatile = True)
        output = model(input_var)
        output_mean = output.view(batch_size, n_crops, -1).mean(1)
        pred = torch.cat((pred, output_mean.data), 0)
    
    aurocs = compute_AUCs(ground_truth, pred)
    aurocs_average = np.array(aurocs).mean()
    print("Average AUROC = ", aurocs_average)
    
    for i in range(config["NUM_CLASSES"]) : 
        print(f"AUROC of {config["CLASS_NAMES"][i]} is {aurocs[i]}")

In [19]:
"""
if __name__ == "__main__" : 
    main()
"""

'\nif __name__ == "__main__" : \n    main()\n'